# DNA_classifier에 맞는 bias, weights 모델 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from base import BaseModel
from trainer import *
from model import *
from sklearn.datasets import make_classification
confined_b = Confined_bias.apply
confined_w = Confined_weight.apply

## 3-4-2 node 로 구성된 classifier 생성

In [3]:
layers=[3,4,2]

### 각 맞는 directory에서 model의 best weight 불러오고 확인

In [5]:
model = Model(*layers)
loading = torch.load('saved/models/sklearn_net/clinear_4_2_scaled_bt_1/model_best.pth', map_location='cpu')
sd = loading['state_dict']
model.load_state_dict(sd)
model.eval()

weights={}
biases={}
for i in range(len(layers)-1):
    weights['weight'+str(i+1)]=confined_w(sd['fc'+str(i+1)+'.weight'])
    biases['bias'+str(i+1)]=confined_b(sd['fc'+str(i+1)+'.bias'])
    print(sum(sum(weights['weight'+str(i+1)]!=0)))
    print(weights['weight'+str(i+1)])
    print(sum(biases['bias'+str(i+1)]!=0))
    print(biases['bias'+str(i+1)])
for i in range(len(layers)-1):
    weights['weight'+str(i+1)+'_t']=torch.transpose(weights['weight'+str(i+1)],0,1)
    print(weights['weight'+str(i+1)+'_t'])    
print()

tensor(11)
tensor([[-0.2000,  0.0000, -0.2000],
        [ 0.3000, -0.4000,  0.2000],
        [-0.3000, -0.4000, -0.3000],
        [ 0.7000,  0.7000, -0.3000]])
tensor(2)
tensor([ 0.0000, -0.2260,  0.0000, -0.5100])
tensor(8)
tensor([[-0.4000, -0.3000, -0.2000, -0.7000],
        [-0.4000,  0.3000, -0.1000,  0.7000]])
tensor(1)
tensor([ 0.0000, -0.3390])
tensor([[-0.2000,  0.3000, -0.3000,  0.7000],
        [ 0.0000, -0.4000, -0.4000,  0.7000],
        [-0.2000,  0.2000, -0.3000, -0.3000]])
tensor([[-0.4000, -0.4000],
        [-0.3000,  0.3000],
        [-0.2000, -0.1000],
        [-0.7000,  0.7000]])



## Seesaw compiler로 만든 파일에서 붙여넣어 여러 orthogonal sequence 생성

In [6]:
sequences="""S4	= CATCCATTCCACTCA	S4*	= TGAGTGGAATGGATG
S5	= CACCACCAAACTTCA	S5*	= TGAAGTTTGGTGGTG
S6	= CATAACACAATCACA	S6*	= TGTGATTGTGTTATG
S7	= CACTAACATACAACA	S7*	= TGTTGTATGTTAGTG
S8	= CATACAACATCTACA	S8*	= TGTAGATGTTGTATG
S9	= CATCAATCAACACCA	S9*	= TGGTGTTGATTGATG
S10	= CACAACTCATTACCA	S10*	= TGGTAATGAGTTGTG
S11	= CATCTTCTAACATCA	S11*	= TGATGTTAGAAGATG
S12	= CAATCTAACACTCCA	S12*	= TGGAGTGTTAGATTG
S13	= CAACCATACTAAACA	S13*	= TGTTTAGTATGGTTG
S14	= CATTCCTACATTTCA	S14*	= TGAAATGTAGGAATG
S15	= CAAATCTTCATCCCA	S15*	= TGGGATGAAGATTTG
S16	= CACTCATCCTTTACA	S16*	= TGTAAAGGATGAGTG
S17	= CAATTCACTCAATCA	S17*	= TGATTGAGTGAATTG
S18	= CATTCATTACCTCCA	S18*	= TGGAGGTAATGAATG
S19	= CAAACACTCTATTCA	S19*	= TGAATAGAGTGTTTG
S20	= CATCTACAATTCACA	S20*	= TGTGAATTGTAGATG
S21	= CACCAATACTCCTCA	S21*	= TGAGGAGTATTGGTG
S22	= CACCATTACAATCCA	S22*	= TGGATTGTAATGGTG
S23	= CAATCCACACTTCCA	S23*	= TGGAAGTGTGGATTG
S24	= CACACTTCAAACTCA	S24*	= TGAGTTTGAAGTGTG
S25	= CAACTCAAACATACA	S25*	= TGTATGTTTGAGTTG
S26	= CACATAACAAAACCA	S26*	= TGGTTTTGTTATGTG
S27	= CACTCTCCATCACCA	S27*	= TGGTGATGGAGAGTG
S28	= CAAACTAAACAACCA	S28*	= TGGTTGTTTAGTTTG
S29	= CACCTCTTCCCTTCA	S29*	= TGAAGGGAAGAGGTG
S30	= CATACCCTTTTCTCA	S30*	= TGAGAAAAGGGTATG
S31	= CACCCAAAACCCACA	S31*	= TGTGGGTTTTGGGTG
S32	= CAACTTTCATATCCA	S32*	= TGGATATGAAAGTTG
S33	= CAAACCCAACTCACA	S33*	= TGTGAGTTGGGTTTG
S34	= CATTCTCCCACCTCA	S34*	= TGAGGTGGGAGAATG
S35	= CATCACCACTATACA	S35*	= TGTATAGTGGTGATG
S36	= CATCCTTAACTCCCA	S36*	= TGGGAGTTAAGGATG
S37	= CATTACCAACCACCA	S37*	= TGGTGGTTGGTAATG
S38	= CAATACAAATCCACA	S38*	= TGTGGATTTGTATTG
S39	= CACAAACTACATCCA	S39*	= TGGATGTAGTTTGTG
S40	= CACTTCACAACTACA	S40*	= TGTAGTTGTGAAGTG
S41	= CAACAAACCATTACA	S41*	= TGTAATGGTTTGTTG
S42	= CACATTCAACCTACA	S42*	= TGTAGGTTGAATGTG
S43	= CACATCATATCATCA	S43*	= TGATGATATGATGTG
S44	= CACTTTTCACTATCA	S44*	= TGATAGTGAAAAGTG
S45	= CATTTCCCAAACCCA	S45*	= TGGGTTTGGGAAATG
S46	= CATCCCACATAACCA	S46*	= TGGTTATGTGGGATG
S47	= CAACCTTCTACCACA	S47*	= TGTGGTAGAAGGTTG
S56	= CACTACTAACTTACA	S56*	= TGTAAGTTAGTAGTG
S58	= CATCTCTCCTTATCA	S58*	= TGATAAGGAGAGATG
S60	= CACCTAATCTTTCCA	S60*	= TGGAAAGATTAGGTG
S62	= CACTTACACTAATCA	S62*	= TGATTAGTGTAAGTG"""

In [7]:
splited=sequences.split('\n')
seqs=[]
seqs_p=[]

s=0
for i in range(len(layers)-1):
    s+=layers[i]*layers[i+1]

for i in range(s) :
    tmp, m = splited[i].split('= ')[1:3]
    p = tmp.split('\t')[0]
    seqs.append((p,m))
seqs[:5]

[('CATCCATTCCACTCA', 'TGAGTGGAATGGATG'),
 ('CACCACCAAACTTCA', 'TGAAGTTTGGTGGTG'),
 ('CATAACACAATCACA', 'TGTGATTGTGTTATG'),
 ('CACTAACATACAACA', 'TGTTGTATGTTAGTG'),
 ('CATACAACATCTACA', 'TGTAGATGTTGTATG')]

### validation을 위한 함수 생성

In [35]:
def val_ck(seq, ck, mode='comp'):
    t=''
    for n in seq:
        if n=='A':
            t+='T'
        elif n=='G':
            t+='C'
        elif n=='C':
            t+='G'
        elif n=='T':
            t+='A'
    if mode!='comp':
        t=t[::-1]
    print(t==ck)

def mk_seq(seq, mode='comp'):
    t=''
    for n in seq:
        if n=='A':
            t+='T'
        elif n=='G':
            t+='C'
        elif n=='C':
            t+='G'
        elif n=='T':
            t+='A'
    if mode!='comp':
        t=t[::-1]
    return t

In [11]:
seq_layers=[]
start=0

for i in range(len(layers)): # input, weight strand
    seq_layers.append(seqs[start:start+layers[i]])
    start+=layers[i]
    
for i in range(len(layers)-1): # bias strand
    seq_layers.append(seqs[start:start+layers[i+1]])
    start+=layers[i+1]
    
seq_layers.append(seqs[start:start+layers[-1]]) # output strand(flourescence)
start+=layers[-1]

seq_layers.append([seqs[start]]) # meaningless strand

In [12]:
seq_layers

[[('CATCCATTCCACTCA', 'TGAGTGGAATGGATG'),
  ('CACCACCAAACTTCA', 'TGAAGTTTGGTGGTG'),
  ('CATAACACAATCACA', 'TGTGATTGTGTTATG')],
 [('CACTAACATACAACA', 'TGTTGTATGTTAGTG'),
  ('CATACAACATCTACA', 'TGTAGATGTTGTATG'),
  ('CATCAATCAACACCA', 'TGGTGTTGATTGATG'),
  ('CACAACTCATTACCA', 'TGGTAATGAGTTGTG')],
 [('CATCTTCTAACATCA', 'TGATGTTAGAAGATG'),
  ('CAATCTAACACTCCA', 'TGGAGTGTTAGATTG')],
 [('CAACCATACTAAACA', 'TGTTTAGTATGGTTG'),
  ('CATTCCTACATTTCA', 'TGAAATGTAGGAATG'),
  ('CAAATCTTCATCCCA', 'TGGGATGAAGATTTG'),
  ('CACTCATCCTTTACA', 'TGTAAAGGATGAGTG')],
 [('CAATTCACTCAATCA', 'TGATTGAGTGAATTG'),
  ('CATTCATTACCTCCA', 'TGGAGGTAATGAATG')],
 [('CAAACACTCTATTCA', 'TGAATAGAGTGTTTG'),
  ('CATCTACAATTCACA', 'TGTGAATTGTAGATG')],
 [('CACCAATACTCCTCA', 'TGAGGAGTATTGGTG')]]

In [17]:
# 정수형 weight strand 만드는 방법
strands={}
l=0
write_wb=Workbook()
write_ws = write_wb.active
write_ws['A1'] = 'Input Sequence'

print('Input')
tmp=[]
for i in range(len(seq_layers[0])):
    print("5'"+seq_layers[0][i][0])
    tmp.append(seq_layers[0][i][0])
write_ws.append(tmp)
print('')


sum_=[]
sum_2=[]
for i in range(len(layers)-1):    
    print('weight')
    weight=weights['weight'+str(i+1)+'_t']
    bias=biases['bias'+str(i+1)]
    
    for j in range(weight.size(0)):
        ck=0
        for k in range(weight.size(1)):
            ori_seq_t=' '*15 # empty space
            if i==0:
                ori_seq_b=seq_layers[i][j][1][::-1] # complementary bind site
            else :
                ori_seq_b=seq_layers[i+len(layers)-1][j][1][::-1] # complementary bind site
            
            sum_.append(weight.size(1))
            write_ws['A'+str(3*(j+sum(sum_[:-1]))+3)] = 'Layer'+str(i+1)+' Hidden Sequence '+str(sum(sum_2)+k+1)
            tmp=["5'->3'",'']
            
            ele=weight[j][k]
            sign=0 if ele>=0 else 1
            num=abs(int(10*ele))
            
            if num==0:
                print('zero')
                continue
            
            ck+=1
            add_t=seq_layers[i+1][k][sign]+'/' # switching
            add_b=seq_layers[i+1][k][1-sign][::-1]
            
            for l in range(num):
                tmp.append(add_t[:-1])
            
            ori_seq_t+=(add_t*num)
            ori_seq_b+=((add_b[:-2]+add_t[-3:-1])*(num))
            
            print("  "+ori_seq_t+"3'")
            print("3'"+ori_seq_b)
            
            write_ws.append(tmp)
            write_ws['A'+str(3*(j+sum(sum_[:-1]))+5)]="3'->5'"
            write_ws['B'+str(3*(j+sum(sum_[:-1]))+5)]=ori_seq_b            
            
        sum_2.append(ck)
        print()
        if j==weight.size(0)-1:
            print('\ndivider') # The outputs of hidden layers are less than 10 --> weight.size(1)<10 
            
            for m in range(bias.size(0)):
                d_ori_seq_t=' '*15
                d_ori_seq_b=seq_layers[i+1][m][1][::-1] # complementary bind site
                                
                # Changing & Division
                
                conc_ratio=10/weight.size(1)-1
                print("  "+d_ori_seq_t+seq_layers[-1][0][0])
                print("3'"+d_ori_seq_b+seq_layers[-1][0][1][::-1], conc_ratio)
                
                print("  "+d_ori_seq_t+seq_layers[i+len(layers)][m][0])
                print("3'"+d_ori_seq_b+seq_layers[i+len(layers)][m][1][::-1], 1)
                print('\n')
            
            print('bias')
            for m in range(bias.size(0)):
                b_ori_seq=seq_layers[i+len(layers)][m][1][::-1] # complementary bind site
                conc=abs(bias[m])
                if conc==0:
                    print('zero')
                    continue
                print("3'"+b_ori_seq, conc)        
            print('\n')
            
    if i==len(layers)-2:
        print('output')
        for m in range(layers[-1]):
            o_ori_seq_t=' '*15+seq_layers[i+len(layers)+1][m][0]
            o_ori_seq_b=seq_layers[i+len(layers)][m][1][::-1]+seq_layers[i+len(layers)+1][m][1][::-1]
            print("  "+o_ori_seq_t+"3'-F")
            print("3'"+o_ori_seq_b+'-Q')
        
    print('\n')

write_wb.save('DNA_sequences.xlsx')

Input
5'CATCCATTCCACTCA
5'CACCACCAAACTTCA
5'CATAACACAATCACA

weight
                 TGTTGTATGTTAGTG/TGTTGTATGTTAGTG/3'
3'GTAGGTAAGGTGAGTACAACATACAATCTGACAACATACAATCTG
                 CATACAACATCTACA/CATACAACATCTACA/CATACAACATCTACA/3'
3'GTAGGTAAGGTGAGTGTATGTTGTAGATCAGTATGTTGTAGATCAGTATGTTGTAGATCA
                 TGGTGTTGATTGATG/TGGTGTTGATTGATG/TGGTGTTGATTGATG/3'
3'GTAGGTAAGGTGAGTACCACAACTAACTTGACCACAACTAACTTGACCACAACTAACTTG
                 CACAACTCATTACCA/CACAACTCATTACCA/CACAACTCATTACCA/CACAACTCATTACCA/CACAACTCATTACCA/CACAACTCATTACCA/CACAACTCATTACCA/3'
3'GTAGGTAAGGTGAGTGTGTTGAGTAATGCAGTGTTGAGTAATGCAGTGTTGAGTAATGCAGTGTTGAGTAATGCAGTGTTGAGTAATGCAGTGTTGAGTAATGCAGTGTTGAGTAATGCA

zero
                 TGTAGATGTTGTATG/TGTAGATGTTGTATG/TGTAGATGTTGTATG/TGTAGATGTTGTATG/3'
3'GTGGTGGTTTGAAGTACATCTACAACATTGACATCTACAACATTGACATCTACAACATTGACATCTACAACATTG
                 TGGTGTTGATTGATG/TGGTGTTGATTGATG/TGGTGTTGATTGATG/TGGTGTTGATTGATG/3'
3'GTGGTGGTTTGAAGTACCACAACTAACTTGACCACAACTAACTTGACCACAACTAACTTGA

In [ ]:
# 정수형 weight strand 만드는 방법
strands={}
l=0
write_wb=Workbook()
write_ws = write_wb.active
write_ws['A1'] = 'Input Sequence'

print('Input')
tmp=[]
for i in range(len(seq_layers[0])):
    print("5'"+seq_layers[0][i][0])
    tmp.append(seq_layers[0][i][0])
write_ws.append(tmp)
print('')


sum_=[]
sum_2=[]
for i in range(len(layers)-1):    
    print('weight')
    weight=weights['weight'+str(i+1)+'_t']
    bias=biases['bias'+str(i+1)]
    
    for j in range(weight.size(0)):
        ck=0
        for k in range(weight.size(1)):
            ori_seq_t=' '*15 # empty space
            if i==0:
                ori_seq_b=seq_layers[i][j][1][::-1] # complementary bind site
            else :
                ori_seq_b=seq_layers[i+len(layers)-1][j][1][::-1] # complementary bind site
            
            sum_.append(weight.size(1))
            write_ws['A'+str(3*(j+sum(sum_[:-1]))+3)] = 'Layer'+str(i+1)+' Hidden Sequence '+str(sum(sum_2)+k+1)
            tmp=["5'->3'",'']
            
            ele=weight[j][k]
            sign=0 if ele>=0 else 1
            num=abs(int(10*ele))
            
            if num==0:
                print('zero')
                continue
            
            ck+=1
            add_t=seq_layers[i+1][k][sign]+'/' # switching
            add_b=seq_layers[i+1][k][1-sign][::-1]
            
            for l in range(num):
                tmp.append(add_t[:-1])
            
            ori_seq_t+=(add_t*num)
            ori_seq_b+=((add_b[:-2]+add_t[-3:-1])*(num))
            
            print("  "+ori_seq_t+"3'")
            print("3'"+ori_seq_b)
            
            write_ws.append(tmp)
            write_ws['A'+str(3*(j+sum(sum_[:-1]))+5)]="3'->5'"
            write_ws['B'+str(3*(j+sum(sum_[:-1]))+5)]=ori_seq_b            
            
        sum_2.append(ck)
        print()
        if j==weight.size(0)-1:
            print('\ndivider') # The outputs of hidden layers are less than 10 --> weight.size(1)<10  
            for m in range(bias.size(0)):
                if m==bias.size(0)-1:
                    d_ori_seq_t=' '*15
                    d_ori_seq_b=seq_layers[i+1][m][0][::-1] # bind site
                else :
                    d_ori_seq_t=' '*15
                    d_ori_seq_b=seq_layers[i+1][m][1][::-1] # complementary bind site
                                
                # Changing & Division
                
                conc_ratio=10/weight.size(1)-1
                print("  "+d_ori_seq_t+seq_layers[-1][0][0])
                print("3'"+d_ori_seq_b+seq_layers[-1][0][1][::-1], conc_ratio)
                
                print("  "+d_ori_seq_t+seq_layers[i+len(layers)][m][0])
                print("3'"+d_ori_seq_b+seq_layers[i+len(layers)][m][1][::-1], 1)
                print('\n')
            
            print('bias')
            for m in range(bias.size(0)):
                if m==bias.size(0)-1:
                    b_ori_seq=seq_layers[i+len(layers)][m][0][::-1] # bind site
                else :
                    b_ori_seq=seq_layers[i+len(layers)][m][1][::-1] # complementary bind site
                conc=abs(bias[m])
                if conc==0:
                    print('zero')
                    continue
                print("3'"+b_ori_seq, conc)        
            print('\n')
            
    if i==len(layers)-2:
        print('output')
        for m in range(layers[-1]):
            o_ori_seq_t=' '*15+seq_layers[i+len(layers)+1][m][0]
            o_ori_seq_b=seq_layers[i+len(layers)][m][1][::-1]+seq_layers[i+len(layers)+1][m][1][::-1]
            print("  "+o_ori_seq_t+"3'-F")
            print("3'"+o_ori_seq_b+'-Q')
        
    print('\n')

write_wb.save('DNA_sequences.xlsx')

In [34]:
# val_ck('CAATCTAACACTCCA','GTTAGATTGTGAGGT', 'comp')
val_ck('CATCTTCTAACATCA', 'TGATGTTAGAAGATG')

True

In [57]:
for i in range(len(layers)-1):
    print(weights['weight'+str(i+1)+'_t'])

tensor([[-0.2000,  0.3000, -0.3000,  0.7000],
        [ 0.0000, -0.4000, -0.4000,  0.7000],
        [-0.2000,  0.2000, -0.3000, -0.3000]])
tensor([[-0.4000, -0.4000],
        [-0.3000,  0.3000],
        [-0.2000, -0.1000],
        [-0.7000,  0.7000]])


In [37]:
print("""Input
5'CAACCATACTAAACA3'
""")
print("""weight 1
                 CATCTTCTAACATCA/CATCTTCTAACATCA
3'GTTGGTATGATTTGTGTAGAAGATTGTACAGTAGAAGATTGTACA
""")

print("""weight 2
                 TGATGTTAGAAGATG
3'GTTGGTATGATTTGTACTACAATCTTCTTG
""")

print("""divider
                 CAATTCACTCAATCA
3'GTAGAAGATTGTAGTGTTAAGTGAGTTAGT 1
""")
                
print("""output
                 CAAACACTCTATTCA/3'-F
3'GTTAAGTGAGTTAGTGTTTGTGAGATAAGT-Q
""")
print('F=ROX')
print('Q=BHQ-2')

Input
5'CAACCATACTAAACA3'

weight 1
                 CATCTTCTAACATCA/CATCTTCTAACATCA
3'GTTGGTATGATTTGTGTAGAAGATTGTACAGTAGAAGATTGTACA

weight 2
                 TGATGTTAGAAGATG
3'GTTGGTATGATTTGTACTACAATCTTCTTG

divider
                 CAATTCACTCAATCA
3'GTAGAAGATTGTAGTGTTAAGTGAGTTAGT 1

output
                 CAAACACTCTATTCA/3'-F
3'GTTAAGTGAGTTAGTGTTTGTGAGATAAGT-Q

F=ROX
Q=BHQ-2


In [39]:
s="GTTGGTATGATTTGTGTAGAAGATTGTACAGTAGAAGATTGTACA"
print(s[::-1])
s2="GTTGGTATGATTTGTACTACAATCTTCTTG"
print(s2[::-1])

ACATGTTAGAAGATGACATGTTAGAAGATGTGTTTAGTATGGTTG
GTTCTTCTAACATCATGTTTAGTATGGTTG


In [40]:
print("""5'
CAACCATACTAAACA

CATCTTCTAACATCA
ACATGTTAGAAGATGACATGTTAGAAGATGTGTTTAGTATGGTTG

TGATGTTAGAAGATG
GTTCTTCTAACATCATGTTTAGTATGGTTG

CAATTCACTCAATCA
TGATTGAGTGAATTGTGATGTTAGAAGATG

CAAACACTCTATTCA-ROX
BHQ-2-TGAATAGAGTGTTTGTGATTGAGTGAATTG
    """)

5'
CAACCATACTAAACA

CATCTTCTAACATCA
ACATGTTAGAAGATGACATGTTAGAAGATGTGTTTAGTATGGTTG

TGATGTTAGAAGATG
GTTCTTCTAACATCATGTTTAGTATGGTTG

CAATTCACTCAATCA
TGATTGAGTGAATTGTGATGTTAGAAGATG

CAAACACTCTATTCA-ROX
BHQ-2-TGAATAGAGTGTTTGTGATTGAGTGAATTG
    
